## test netcdf+

This is a more extensive integration test, if all the features of netcdf+ work as expected.

In [1]:
import openpathsampling as paths
from openpathsampling.netcdfplus import (
    NetCDFPlus, 
    ObjectStore, 
    StorableObject,
    NamedObjectStore,
    UniqueNamedObjectStore,
    DictStore,
    ImmutableDictStore,
    VariableStore,
    StorableNamedObject
)
from uuid import UUID
import numpy as np

In [2]:
class Node(StorableObject):
    def __init__(self, value):
        super(Node, self).__init__()
        self.value = value
        
    def __repr__(self):
        return 'Node(%s)' % self.value

In [3]:
class NamedNode(StorableNamedObject):
    def __init__(self, value):
        super(NamedNode, self).__init__()
        self.value = value
        
    def __repr__(self):
        return 'Node(%s)' % self.value

In [4]:
class NullNode(StorableObject):        
    _json_store_name = 'nullnodes'  # this should enable autocreation
    def __repr__(self):
        return 'NullNode()'

### Open new storage

try to create a new storage

In [5]:
st = NetCDFPlus('test_netcdfplus.nc', mode='w')

### Create some stores

In [6]:
class NodeIntStore(VariableStore):
    def __init__(self):
        super(NodeIntStore, self).__init__(Node, ['value'])
    
    def initialize(self):
        super(VariableStore, self).initialize()

        # Add here the stores to be imported
        self.create_variable('value', 'int')    

In [7]:
st.add_store('nodesnamed', NamedObjectStore(NamedNode))
st.add_store('nodesunique', UniqueNamedObjectStore(NamedNode))
st.add_store('dict', DictStore())
st.add_store('dictimmutable', ImmutableDictStore())
st.add_store('varstore', NodeIntStore())

And the default store. The last store for a particular object is used as the default if no specific store is specified.

In [8]:
print st.add_class_json_store('nodes', Node)

store.nodes[Node] : 0 object(s)


Test automatic creation of store

In [9]:
ob = NullNode()

This should be storable and 

In [10]:
print st.save(ob)

(store.nullnodes[NullNode] : 1 object(s), 6, 0)


In [11]:
print st.find_store(Node)
print st.find_store(NullNode)

store.nodes[Node] : 0 object(s)
store.nullnodes[NullNode] : 1 object(s)


Initialize the store

In [12]:
st.nodes.save(Node(10));

In [13]:
st.close()

### Reopen empty storage

In [14]:
st = NetCDFPlus('test_netcdfplus.nc', mode='a')

set caching of the new stores

In [15]:
for store in st.stores:
    store.set_caching(10)

Check if the stores were correctly loaded

In [16]:
assert('nodes' in st.objects)

In [17]:
assert('stores' in st.objects)

In [18]:
assert(len(st.nodes) == 1)

In [19]:
assert(len(st.stores) == 7)

In [20]:
for store in st.stores:
    print '{:40} {:30}'.format(store, store.cache)

store.nodesnamed[NamedNode] : 0 object(s) WeakLRUCache(0/0 of 10/Inf)   
store.nodesunique[NamedNode] : 0 object(s) WeakLRUCache(0/0 of 10/Inf)   
store.dict[None/ANY] : 0 object(s)       WeakLRUCache(0/0 of 10/Inf)   
store.dictimmutable[None/ANY] : 0 object(s) WeakLRUCache(0/0 of 10/Inf)   
store.varstore[Node] : 0 object(s)       WeakLRUCache(0/0 of 10/Inf)   
store.nodes[Node] : 1 object(s)          WeakLRUCache(0/0 of 10/Inf)   
store.nullnodes[NullNode] : 1 object(s)  WeakLRUCache(0/0 of 10/Inf)   


### Create variables types

Get a list of all possible variable types

In [21]:
print sorted(st.get_var_types())

['bool', 'float', 'index', 'int', 'json', 'jsonobj', 'lazyobj', u'lazyobj.dict', u'lazyobj.dictimmutable', u'lazyobj.nodes', u'lazyobj.nodesnamed', u'lazyobj.nodesunique', u'lazyobj.nullnodes', 'lazyobj.stores', u'lazyobj.varstore', 'length', 'long', 'numpy.float32', 'numpy.float64', 'numpy.int16', 'numpy.int32', 'numpy.int64', 'numpy.int8', 'numpy.uint16', 'numpy.uint32', 'numpy.uint64', 'numpy.uint8', 'obj', u'obj.dict', u'obj.dictimmutable', u'obj.nodes', u'obj.nodesnamed', u'obj.nodesunique', u'obj.nullnodes', 'obj.stores', u'obj.varstore', 'store', 'str', 'uuid']


Make a dimension on length 2 to simplify dimension nameing.

Now we construct for each type a corresponding variable of dimensions 2x2x2.

In [22]:
st.create_dimension('pair', 2)

In [23]:
for var_type in st.get_var_types():
    st.create_variable(var_type, var_type, dimensions=('pair', 'pair', 'pair'))

In [24]:
st.update_delegates()

In [25]:
for var_name, var in sorted(st.variables.items()):
    print var_name, var.dimensions

bool (u'pair', u'pair', u'pair')
dict_json (u'dict',)
dict_name (u'dict',)
dictimmutable_json (u'dictimmutable',)
dictimmutable_name (u'dictimmutable',)
float (u'pair', u'pair', u'pair')
index (u'pair', u'pair', u'pair')
int (u'pair', u'pair', u'pair')
json (u'pair', u'pair', u'pair')
jsonobj (u'pair', u'pair', u'pair')
lazyobj (u'pair', u'pair', u'pair', u'pair')
lazyobj.dict (u'pair', u'pair', u'pair')
lazyobj.dictimmutable (u'pair', u'pair', u'pair')
lazyobj.nodes (u'pair', u'pair', u'pair')
lazyobj.nodesnamed (u'pair', u'pair', u'pair')
lazyobj.nodesunique (u'pair', u'pair', u'pair')
lazyobj.nullnodes (u'pair', u'pair', u'pair')
lazyobj.stores (u'pair', u'pair', u'pair')
lazyobj.varstore (u'pair', u'pair', u'pair')
length (u'pair', u'pair', u'pair')
long (u'pair', u'pair', u'pair')
nodes_json (u'nodes',)
nodesnamed_json (u'nodesnamed',)
nodesnamed_name (u'nodesnamed',)
nodesunique_json (u'nodesunique',)
nodesunique_name (u'nodesunique',)
nullnodes_json (u'nullnodes',)
numpy.float32

In [26]:
for var in sorted(st.vars):
    print var

bool
dict_json
dict_name
dictimmutable_json
dictimmutable_name
float
index
int
json
jsonobj
lazyobj
lazyobj.dict
lazyobj.dictimmutable
lazyobj.nodes
lazyobj.nodesnamed
lazyobj.nodesunique
lazyobj.nullnodes
lazyobj.stores
lazyobj.varstore
length
long
nodes_json
nodesnamed_json
nodesnamed_name
nodesunique_json
nodesunique_name
nullnodes_json
numpy.float32
numpy.float64
numpy.int16
numpy.int32
numpy.int64
numpy.int8
numpy.uint16
numpy.uint32
numpy.uint64
numpy.uint8
obj
obj.dict
obj.dictimmutable
obj.nodes
obj.nodesnamed
obj.nodesunique
obj.nullnodes
obj.stores
obj.varstore
store
stores_json
stores_name
str
uuid
varstore_value


#### Bool

In [27]:
st.vars['bool'][:] = True

In [28]:
print st.vars['bool'][:]

[[[True, True], [True, True]], [[True, True], [True, True]]]


#### Float

In [29]:
st.vars['float'][1,1] = 1.0

In [30]:
print st.vars['float'][:]

[[[None, None], [None, None]], [[None, None], [1.0, 1.0]]]


#### Index
`Index` is special in the sense that it supports only integers that are non-negative. Negative values will be interpreted as `None`

In [31]:
st.vars['index'][0,1,0] = 10
st.vars['index'][0,1,1] = -1
st.vars['index'][0,0] = None

In [32]:
print st.vars['index'][0,1]
print st.vars['index'][0,0]

[10, None]
[None, None]


#### Int

In [33]:
st.vars['int'][0,1,0] = 10
st.vars['int'][0,1,1] = -1

In [34]:
print st.vars['int'][:]

[[[None, None], [10, -1]], [[None, None], [None, None]]]


#### JSON

The variable type JSON encode the given object as a JSON string in the shortest possible way. This includes using referenes to storable objects. 

In [35]:
st.vars['json'][0,1,1] = {'Hallo': 2, 'Test': 3}

In [36]:
print st.vars['json'][0,1,1]

{'Test': 3, 'Hallo': 2}


In [37]:
st.vars['json'][0,1,0] = Node(10)

In [38]:
print st.variables['json'][0,1,:]

[u'{"_store":"nodes","_idx":1}' u'{"Test":3,"Hallo":2}']


All object types registered as being Storable by subclassing from `openpathsampling.base.StorableObject`.

#### JSONObj

A JSON serializable object. This can be normal very simple python objects, plus numpy arrays, and objects that implement `to_dict` and `from_dict`. This is almost the same as _JSON_ except if the object to be serialized is a storable object itself, it will not be referenced but the object itself will be turned into a JSON representation.

In [39]:
nn = Node(10)
st.vars['jsonobj'][1,0,0] = nn

In [40]:
print st.variables['jsonobj'][1,0,0]

{"_cls":"Node","_dict":{"value":10}}


#### Numpy

In [41]:
st.vars['numpy.float32'][:] = np.ones((2,2,2)) * 3.0
st.vars['numpy.float32'][0] = np.ones((2,2)) * 7.0

In [42]:
print st.vars['numpy.float32'][:]

[[[ 7.  7.]
  [ 7.  7.]]

 [[ 3.  3.]
  [ 3.  3.]]]


#### Obj

You can store objects of a type which you have previously added. For loading you need to make sure that the class (and the store if set manually) is present when you load from the store.

In [43]:
st.vars['obj.nodes'][0,0,0] = Node(1)
st.vars['obj.nodes'][0,1,0] = Node('Second')
st.vars['obj.nodes'][0,0,1] = Node('Third')

In some cases we can also assign one element to a group of elements like it is possible for numbers.

In [44]:
st.vars['obj.nodes'][1] = Node(20)

In [45]:
print st.variables['obj.nodes'][:]
print st.variables['nodes_json'][:]

[[[2 4]
  [3 --]]

 [[5 5]
  [5 5]]]
[u'{"_cls":"Node","_dict":{"value":10}}'
 u'{"_cls":"Node","_dict":{"value":10}}'
 u'{"_cls":"Node","_dict":{"value":1}}'
 u'{"_cls":"Node","_dict":{"value":"Second"}}'
 u'{"_cls":"Node","_dict":{"value":"Third"}}'
 u'{"_cls":"Node","_dict":{"value":20}}']


In [46]:
print st.vars['obj.nodes'][0,0,0]
print type(st.vars['obj.nodes'][0,0,0])

Node(1)
<class '__main__.Node'>


#### lazy

Lazy loading will reconstruct an object using proxies. These proxies behave almost like the loaded object, but will delay loading of the object until it is accessed. Saving for lazy objects is the same as for regular objects. Only loading return a proxy object.

In [47]:
st.vars['lazyobj.nodes'][0,0,0] = Node('First')

The type of the returned object is `LoaderProxy` while the class is the actual class is the baseclass loaded by the store to not trigger loading when the `__class__` attribute is accessed. The actual object can be accessed by `__subject__` and doing so will trigger loading the object. All regular attributes will be delegated to `__subject__.attribute` and also trigger loading.

In [48]:
proxy = st.vars['lazyobj.nodes'][0,0,0]
print 'Type:   ', type(proxy)
print 'Class:  ', proxy.__class__
print 'Content:', proxy.__subject__.__dict__
print 'Access: ', proxy.value

Type:    <class 'openpathsampling.netcdfplus.proxy.LoaderProxy'>
Class:   <class '__main__.Node'>
Content: {'__uuid__': UUID('fd53d4a1-a798-11e6-9f9c-000000000032'), 'value': 'First'}
Access:  First


### Load/Save objects

Note that there are now 6 `Node` objects.

In [49]:
print st.nodes[:]

[Node(10), Node(10), Node(1), Node(Second), Node(Third), Node(20), Node(First)]


In [50]:
obj = Node('BlaBla')
st.nodes.save(obj);

Saving without specifying should use store `nodes` which was defined last.

In [51]:
print len(st.nodes)
obj = Node('BlaBlaBla')
st.save(obj)
print len(st.nodes)

8
9


Get the index of the obj in the storage

In [52]:
print st.idx(obj)

8


And test the different ways to access the contained `json`

#### 1. direct `json` using `variables` in the store

In [53]:
print st.nodes.variables['json'][st.idx(obj)]

{"_cls":"Node","_dict":{"value":"BlaBlaBla"}}


#### 2. direct `json` using `variables` in the full storage

In [54]:
print st.variables['nodes_json'][st.idx(obj)]

{"_cls":"Node","_dict":{"value":"BlaBlaBla"}}


#### 3. indirect `json` and reconstruct using `vars` in the store

In [55]:
print st.nodes.vars['json'][st.idx(obj)]
print st.nodes.vars['json'][st.idx(obj)] is obj

Node(BlaBlaBla)
False


#### 4. using the store accessor  `__getitem__` in the store

In [56]:
print st.nodes[st.idx(obj)]
print st.nodes[st.idx(obj)] is obj

Node(BlaBlaBla)
True


One importance difference is that a store like `nodes` has a cache (which we set to 10 before). Using `vars` will not use a store and hence create a new object!

### ObjectStores

ObjectStores are resposible to save and load objects. There are now 6 types available.

#### ObjectStore

The basic store which we have used before

#### NamedObjectStore

Supports to give objects names

In [57]:
n = NamedNode(3)

NamedObjects have a `.name` property, which has a default.

In [58]:
print n.name

[NamedNode]


and can be set.

In [59]:
n.name = 'OneNode'
print n.name
n.name = 'MyNode'
print n.name

OneNode
MyNode


Once the object is saved, the name cannot be changed anymore.

In [60]:
st.nodesnamed.save(n);

In [61]:
try:
    n.name = 'NewName'
except ValueError as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

# We had an exception
Objects cannot be renamed to `NewName` after is has been saved, it is already named `MyNode`


usually names are not unique (see next store). So we can have more than one object with the same name.

In [62]:
n2 = NamedNode(9)
n2.name = 'MyNode'
st.nodesnamed.save(n2);

See the list of named objects

In [63]:
print st.nodesnamed.name_idx

{'MyNode': set([0, 1])}


#### UniqueNamedObjectStore

The forces names to be unique

In [64]:
st.nodesunique.save(n);

Note here that an object can be store more than once in a storage, but only if more than one store supports the file type.

In [65]:
try:
    st.nodesunique.save(n2)
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')


# We had an exception
Current name "MyNode" is already taken in unique name store. This means you cannot save object "Node(9)" at all. In general this should not happen to unsaved objects unless you fixed the name of the object yourself. Check your code for the generation of objects of the same name.


As said before this can only happen if you have more than one store for the same object type.

In [66]:
print st.nodesunique.name_idx

{'MyNode': set([0])}


some more tests. First saving onnamed objects. This is okay. Only given names should be unique.

In [67]:
n3 = NamedNode(10)
n4 = NamedNode(12)
st.nodesunique.save(n3);
st.nodesunique.save(n4);

In [68]:
n5 = NamedNode(1)
n5.name = 'MyNode'

In [69]:
try:
    st.nodesunique.save(n5)
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

# We had an exception
Current name "MyNode" is already taken in unique name store nodesunique. Try renaming object or saving using other name.


This works since it does a rename before saving.

In [70]:
st.nodesunique.save(n5, 'NextNode');

In [71]:
n6 = NamedNode(1)
n6.name = 'SecondNode'

In [72]:
try:
    st.nodesunique.save(n6, 'MyNode')
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

# We had an exception
New name "MyNode" already taken in unique name store. Try different name instead./nCurrent name "SecondNode" is still free. Saving without giving a specific name should work


#### DictStore

A dictstore works a like a dictionary on disk. The content is returned using `dict()`

In [73]:
print dict(st.dict)
print st.dict.name_idx

{}
{}


In [74]:
n1 = NamedNode(1)
n2 = NamedNode(2)
n3 = NamedNode(3)
st.dict['Number1'] = n1

In [75]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

Number1 : Node(1) [0]


In [76]:
st.dict['Number2'] = n2

In [77]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

Number1 : Node(1) [0]
Number2 : Node(2) [1]


In [78]:
st.dict['Number1'] = n3

In [79]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

Number1 : Node(3) [0, 2]
Number2 : Node(2) [1]


In [80]:
print st.dict['Number1']

Node(3)


In [81]:
print st.dict.find('Number1')

Node(3)


In [82]:
print '[', ', '.join(st.dict.variables['json'][:]), ']'

[ {"_store":"nodesunique","_idx":4}, {"_store":"nodesunique","_idx":5}, {"_store":"nodesunique","_idx":6} ]


In [83]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

Number1 : Node(3) [0, 2]
Number2 : Node(2) [1]


#### ImmutableDictStore

This adds the check that already used names cannot be used again

In [84]:
try:
    st.dictimmutable['Number1'] = n1
    st.dictimmutable['Number1'] = n2
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

# We had an exception
Cannot re-save existing key "Number1" in immutable dict store.


#### VariableStore

Store a node with an int as we defined for our `VariableStore`

In [85]:
a = Node(30)
st.varstore.save(a);

clear the cache

In [86]:
st.varstore.clear_cache()

And try loading

In [87]:
assert(st.varstore[0].value == 30)

Try storing non int() parseable value

In [88]:
try:
    a = Node('test')
    print st.varstore.save(a)
except ValueError as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

# We had an exception
invalid literal for int() with base 10: 'test'
